<a href="https://colab.research.google.com/github/Nurkic/Nurkic/blob/master/baseline_seti_kn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [2]:
%%bash

mkdir -p /root/.kaggle
cp -f ./drive/MyDrive/kaggle/kaggle.json /root/.kaggle/
chmod 600 /root/.kaggle/kaggle.json
pip install -U kaggle

mkdir -p /root/.ssh/
cd drive/MyDrive/kaggle
cp id_rsa id_rsa.pub known_hosts /root/.ssh/

In [3]:
!ls

drive  sample_data


In [4]:
%cd drive/MyDrive/kaggle

/content/drive/MyDrive/kaggle


In [5]:
! pip install -q pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

In [6]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns
import gc
import torch
print(torch.__version__)

1.9.0+cu102


In [7]:
!ls

id_rsa	     known_hosts      sample_submission.csv	    train.log
id_rsa.pub   old_leaky_data   seti-breakthrough-listen.zip
input	     output	      test
kaggle.json  outputtrain.log  train


In [8]:
train = pd.read_csv('./input/seti-breakthrough-listen/train_labels.csv')
test = pd.read_csv('./input/seti-breakthrough-listen/sample_submission.csv')

def get_train_file_path(image_id):
    return "../input/seti-breakthrough-listen/train/{}/{}.npy".format(image_id[0], image_id)

def get_test_file_path(image_id):
    return "../input/seti-breakthrough-listen/test/{}/{}.npy".format(image_id[0], image_id)

train['file_path'] = train['id'].apply(get_train_file_path)
test['file_path'] = test['id'].apply(get_test_file_path)

display(train.head())
display(test.head())

,id,target,file_path
0,0000799a2b2c42d,0,../input/seti-breakthrough-listen/train/0/0000...
1,00042890562ff68,0,../input/seti-breakthrough-listen/train/0/0004...
2,0005364cdcb8e5b,0,../input/seti-breakthrough-listen/train/0/0005...
3,0007a5a46901c56,0,../input/seti-breakthrough-listen/train/0/0007...
4,0009283e145448e,0,../input/seti-breakthrough-listen/train/0/0009...


,id,target,file_path
0,000bf832cae9ff1,0.5,../input/seti-breakthrough-listen/test/0/000bf...
1,000c74cc71a1140,0.5,../input/seti-breakthrough-listen/test/0/000c7...
2,000f5f9851161d3,0.5,../input/seti-breakthrough-listen/test/0/000f5...
3,000f7499e95aba6,0.5,../input/seti-breakthrough-listen/test/0/000f7...
4,00133ce6ec257f9,0.5,../input/seti-breakthrough-listen/test/0/00133...


In [9]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './output'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [10]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    print_freq=100
    num_workers=4
    model_name='efficientnet_b0'  #'efficientnet_b0', 'vit_base_patch16_224', 'tf_efficientnet_b4_ns'
    input_size=512   #512, 768, 1028, 'original'
    output_size=256 
    scheduler='CosineAnnealingLR' #'ReduceLROnPlateau', 'CosineAnnealingLR'
    epochs=25
    factor=0.2 # ReduceLROnPlateau
    patience=4 # ReduceLROnPlateau
    eps=1e-6 # ReduceLROnPlateau
    T_max=15 # CosineAnnealingLR
    lr=1e-4
    min_lr=1e-6
    batch_size=20
    optimizer='adamw'   #adamw', 'adam' 
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_folds=[0]
    train=True
    mode='spatial_3'   #'channel_3', 'channel_6', 'spatial_3', spatial_6'
    aug_mode='mixup'   #'mixup', 'fmix'
    warmup_epochs=2
    multiplier=10
    epoch_no_aug=0
    
if CFG.debug:
    CFG.epochs=3
    train=train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)   

In [11]:

# ====================================================
# Library
# ====================================================
import sys
sys.path.append('./input/image-fmix/FMix-master')
sys.path.append('./input/pytorch-image-models/pytorch-image-models-master')
#print(sys.path)
! pip install --upgrade --force-reinstall --no-deps albumentations
! pip install timm
from fmix import sample_mask

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations import ImageOnlyTransform

import timm
from warmup_scheduler import GradualWarmupScheduler
from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  Using cached albumentations-1.0.3-py3-none-any.whl (98 kB)
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.0.3
    Uninstalling albumentations-1.0.3:
      Successfully uninstalled albumentations-1.0.3


In [12]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = roc_auc_score(y_true, y_pred)
    return score

def init_logger(log_file=OUTPUT_DIR+'train.log'):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = init_logger()

def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [13]:
display("len(train): "+str(len(train)))
display("len(target==0): "+str(len(train[train["target"]==0])))
display("len(target==1): "+str(len(train[train["target"]==1])))



'len(train): 60000'

'len(target==0): 54000'

'len(target==1): 6000'

In [14]:
train_1 = train[train["target"]==1]
train_0 = train[train["target"]==0].sample(n=len(train_1))
beta = len(train_1)/len(train)
train = pd.concat([train_1, train_0]).reset_index()
Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_col])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby(['fold', 'target']).size())

fold  target
0     0         1200
      1         1200
1     0         1200
      1         1200
2     0         1200
      1         1200
3     0         1200
      1         1200
4     0         1200
      1         1200
dtype: int64

In [15]:
def probability_calibration(y_proba, beta):
    """サンプリングレートを元に確率を補正する"""
    calibrated_proba = y_proba / (y_proba + (1 - y_proba) / beta)
    return calibrated_proba

In [16]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df, mode=CFG.mode, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df[CFG.target_col].values
        self.transform = transform
        self.mode = mode
        

        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx][1:]
        label = self.labels[idx]
        image = np.load(file_path)
        
        if self.mode in ['spatial_3', 'channel_3']:
            image = image[::2]
            image = image.astype(np.float32)
        else:
            image = image.astype(np.float32)
        if self.mode in ['spatial_3', 'spatial_6']: 
            image = np.vstack(image).transpose((1, 0))
        elif self.mode in ['channel_3', 'channel_6']:
            image = np.transpose(image, (1,2,0))
            
        if self.transform:
            image = self.transform(image=image)['image']
        else:
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()
            
        label = torch.tensor(label).float()
        return image, label
    
class TestDataset(Dataset):
    def __init__(self, df, mode=CFG.mode, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.transform = transform
        self.mode = mode
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_path = self.file_names[idx][1:]
        image = np.load(file_path)
        if self.mode in ['spatial_3', 'channel_3']:
            image = image[::2].astype(np.float32)
        else:
            image = image.astype(np.float32)
        if self.mode in ['spatial_3', 'spatial_6']:
            image = np.vstack(image).transpose((1, 0))
        elif self.mode in ['channel_3', 'channel_6']:
            image = np.transpose(image, (1,2,0))
        if self.transform:
            image = self.transform(image=image)['image']
        else:
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()
        return image
    
def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

In [17]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    if data == 'train':
        if type(CFG.input_size) == int:
            return A.Compose([
                   A.Resize(CFG.input_size, CFG.input_size),
                   A.HorizontalFlip(p=.5),
                   A.VerticalFlip(p=.5),
                   A.ShiftScaleRotate(rotate_limit=0, p=.25),
                   A.MotionBlur(p=.2),
                   A.IAASharpen(p=.25),
                   ToTensorV2(),
            ])
        else:
            return A.Compose([
                   A.HorizontalFlip(p=.5),
                   A.VerticalFlip(p=.5),
                   A.ShiftScaleRotate(rotate_limit=0, p=.25),
                   A.MotionBlur(p=.2),
                   A.IAASharpen(p=.25),
                   ToTensorV2(),
            ])
            
    elif data == 'valid':
        if type(CFG.input_size) == int:
            return A.Compose([
                   A.Resize(CFG.input_size, CFG.input_size),
                   ToTensorV2(),
            ])
        else:
            return A.Compose([
                   ToTensorV2(),
            ])

In [18]:
# https://github.com/facebookresearch/mixup-cifar10/blob/master/train.py
def mixup_data(x, y, alpha=1, use_cuda=True):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)
    mixed_x = (lam**.5) * x + ((1 - lam)**.5) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def fmix_data(data, targets, alpha=1.0, 
              decay_power=3.0,
              max_soft=0.0,
              shape=(CFG.input_size, CFG.input_size),):
    lam, mask = sample_mask(alpha, decay_power, shape, max_soft)
    indices = torch.randperm(data.size(0)).cuda()
    shuffled_data = data[indices]
    
    targets_a = targets
    targets_b = targets[indices]
    x1 = torch.from_numpy(mask).float()*data
    x2 = torch.from_numpy(1-mask).float()*shuffled_data
    return (x1+x2), targets_a, targets_b, lam

def aug_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [19]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.n = 16
        self.slope = .1
        self.r = 1
        
        if self.cfg.mode in ['spatial_3', 'spatial_6']: 
            self.cnn = timm.create_model(self.cfg.model_name, pretrained=pretrained, in_chans=1)
        elif self.cfg.mode == 'channel_3':
            self.cnn = timm.create_model(self.cfg.model_name, pretrained=pretrained, in_chans=3)
        elif self.cfg.mode == 'channel_6':
            self.cnn = timm.create_model(self.cfg.model_name, pretrained=pretrained, in_chans=6)
        if hasattr(self.cnn, "fc"):
            nb_ft = self.cnn.fc.in_features
            self.cnn.fc = nn.Identity()
        elif hasattr(self.cnn, "_fc"):
            nb_ft = self.cnn._fc.in_features
            self.cnn._fc = nn.Identity()
        elif hasattr(self.cnn, "classifier"):
            nb_ft = self.cnn.classifier.in_features
            self.cnn.classifier = nn.Identity()
        elif hasattr(self.cnn, "last_linear"):
            nb_ft = self.cnn.last_linear.in_features
            self.cnn.last_linear = nn.Identity()
        elif hasattr(self.cnn, "head"):
            nb_ft = self.cnn.head.in_features
            self.cnn.head = nn.Identity()
        
        self.block1 = nn.Sequential(
                nn.Conv2d(1, self.n, kernel_size=(7, 7), stride=(1,1), padding=(1, 1), bias=False),
                nn.LeakyReLU(negative_slope=self.slope),
                nn.Conv2d(self.n, self.n, kernel_size=(1, 1), stride=(1,1), padding=(1, 1), bias=False),
                nn.LeakyReLU(negative_slope=self.slope),
                nn.BatchNorm2d(self.n))
        self.block2 = nn.Sequential(
                nn.Conv2d(self.n, self.n, kernel_size=(3, 3), stride=(1,1), padding=(1, 1), bias=False),
                nn.BatchNorm2d(self.n),
                nn.LeakyReLU(negative_slope=self.slope),
                nn.Conv2d(self.n, self.n, kernel_size=(3, 3), stride=(1,1), padding=(1, 1), bias=False),
                nn.BatchNorm2d(self.n))
        self.block3 = nn.Sequential(
                nn.Conv2d(self.n, self.n, kernel_size=(3, 3), stride=(1,1), padding=(1, 1), bias=False),
                nn.BatchNorm2d(self.n))
        self.block4 = nn.Sequential(
                nn.Conv2d(self.n, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False))
        self.fc = nn.Linear(nb_ft, self.cfg.target_size)

    def forward(self, x):
        res1 = F.interpolate(x, size=(self.cfg.output_size, self.cfg.output_size), mode='bilinear')
        x = self.block1(x)
        res2 = F.interpolate(x, size=(self.cfg.output_size, self.cfg.output_size), mode='bilinear')

        x = self.block2(res2)
        x += res2
        if self.r > 1:
            for _ in range(self.r):
                res2 = x
                x = self.block2(x)
                x += res2
 
        x = self.block3(x)
        x += res2
        
        x = self.block4(x)
        x += res1
        
        x = self.cnn(x)
        x = self.fc(x)
        return x

In [20]:
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

In [21]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device, mode):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (images, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        if mode == 'mixup':
            images, targets_a, targets_b, lam = mixup_data(images, labels.view(-1, 1), use_cuda=True)
            images = images.to(device)
            targets_a = targets_a.to(device)
            targets_b = targets_b.to(device)
            y_preds = model(images)
            loss = aug_criterion(criterion, y_preds, targets_a, targets_b, lam)
            
        elif mode == 'fmix':
            images, targets_a, targets_b, lam = fmix_data(images, labels.view(-1, 1))
            images = images.to(device)
            targets_a = targets_a.to(device)
            targets_b = targets_b.to(device)
            y_preds = model(images)
            loss = aug_criterion(criterion, y_preds, targets_a, targets_b, lam)

        else:
            images = images.to(device)
            labels = labels.to(device)
            y_preds = model(images)
            loss = criterion(y_preds.view(-1), labels)
            
        batch_size = labels.size(0)
        
        # record loss
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  #'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(train_loader)),
                   grad_norm=grad_norm,
                   #lr=scheduler.get_lr()[0],
                   ))
    return losses.avg

def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (images, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(images)
        
        loss = criterion(y_preds.view(-1), labels)
        losses.update(loss.item(), batch_size)
        # record accuracy
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Data {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses,
                   remain=timeSince(start, float(step+1)/len(valid_loader)),
                   ))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

def flip_inference(models, test_loader, device):
    preds = []
    for i, fold in enumerate(CFG.trn_folds):
        model = models[i]
        probs = []
        for step, (images) in tqdm(enumerate(test_loader), total=len(test_loader)):
            images = images.to(device)
            with torch.no_grad():
                y_preds1 = model(images)
                y_preds2 = model(images.flip(-1))
                y_preds3 = model(images.flip(-2))
                y_preds4 = model(images.flip(-1).flip(-2))
            y_preds = (y_preds1.sigmoid().to('cpu').numpy() + y_preds2.sigmoid().to('cpu').numpy() + y_preds3.sigmoid().to('cpu').numpy() + y_preds4.sigmoid().to('cpu').numpy()) / 4
            probs.append(y_preds[:, 0])
        preds.append(np.concatenate(probs))
    return preds

def get_models(CFG, mode='loss'):
    models = []
    for fold in CFG.trn_folds:
        model = CustomModel(cfg=CFG, pretrained=False)
        path = OUTPUT_DIR + f'{CFG.model_name}_fold{fold}_best_{mode}.pth'
        model.load_state_dict(torch.load(path)['model'])
        model.eval()
        model.to(device)
        models.append(model)
    return models

In [22]:
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")
    # ====================================================
    # loader
    # ====================================================
    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index

    train_folds = folds.loc[trn_idx].reset_index(drop=True)
    valid_folds = folds.loc[val_idx].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_col].values

    train_dataset = TrainDataset(train_folds, 
                                 transform=get_transforms(data='train'))
    valid_dataset = TrainDataset(valid_folds, 
                                 transform=get_transforms(data='valid'))
    train_loader = DataLoader(train_dataset, 
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers, pin_memory=True,
                              drop_last=True, worker_init_fn=worker_init_fn)
    valid_loader = DataLoader(valid_dataset, 
                              batch_size=CFG.batch_size * 2, 
                              shuffle=False, 
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)
    # ====================================================
    # optimizer
    # ====================================================
    def get_optimizer(lr=CFG.lr):
        if CFG.optimizer == 'adam':
            optimizer = Adam(model.parameters(), lr=lr, weight_decay=CFG.weight_decay, amsgrad=False)
        elif CFG.optimizer == 'adamw':
            optimizer = AdamW(model.parameters(), lr=lr, weight_decay=CFG.weight_decay)
        return optimizer
    
    # ====================================================
    # criterion
    # ====================================================
    def get_criterion():
        criterion = nn.BCEWithLogitsLoss()
        return criterion
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        if CFG.warmup_epochs > 0:
            scheduler = GradualWarmupSchedulerV2(optimizer, multiplier=CFG.multiplier,
                                                 total_epoch=CFG.warmup_epochs, 
                                                 after_scheduler=scheduler)
        return scheduler 
    
    # ====================================================
    # training loop
    # ====================================================
    model = CustomModel(CFG, pretrained=True)
    model.to(device)
    
    optimizer = get_optimizer()
    scheduler = get_scheduler(optimizer)
    criterion = get_criterion()

    best_score = 0.
    best_loss = np.inf

    for epoch in range(CFG.epochs + CFG.epoch_no_aug):
        start_time = time.time()
        
        # train
        if epoch < CFG.epochs:
            avg_loss = train_fn(train_loader, model, criterion, optimizer,
                                 epoch, scheduler,
                                 device,
                                 mode=CFG.aug_mode)
        else:
            optimizer = get_optimizer(lr=1e-6)
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
            avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device, mode='none')

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        preds = probability_calibration(preds, beta)
    
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, GradualWarmupSchedulerV2):
            scheduler.step()

        # scoring
        score = get_score(valid_labels, preds)

        elapsed = time.time() - start_time
        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')

        if score > best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_score.pth')
        
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.4f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth')
    
    valid_folds['preds'] = torch.load(OUTPUT_DIR+f'{CFG.model_name}_fold{fold}_best_loss.pth', 
                                      map_location=torch.device('cpu'))['preds']

    return valid_folds

In [23]:
# ====================================================
# main
# ====================================================
def main():

    """
    Prepare: 1.train 
    """

    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df[CFG.target_col].values
        score = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}')
    
    if CFG.train:
        # train 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_folds:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        # save result
        oof_df.to_csv(OUTPUT_DIR+'oof_df.csv', index=False)

In [24]:
if __name__ == '__main__':
    main()


========== fold: 0 training ==========


Epoch: [1][0/480] Data 1.027 (1.027) Elapsed 0m 1s (remain 13m 30s) Loss: 0.6938(0.6938) Grad: 2.8114  
Epoch: [1][100/480] Data 0.000 (0.010) Elapsed 0m 34s (remain 2m 8s) Loss: 0.6629(0.6968) Grad: 2.0503  
Epoch: [1][200/480] Data 3.657 (1.903) Elapsed 7m 27s (remain 10m 21s) Loss: 0.6712(0.6986) Grad: 2.6826  
Epoch: [1][300/480] Data 0.000 (3.052) Elapsed 16m 55s (remain 10m 3s) Loss: 0.7109(0.6989) Grad: 2.3410  
Epoch: [1][400/480] Data 0.000 (3.910) Elapsed 28m 16s (remain 5m 34s) Loss: 0.7034(0.6982) Grad: 2.1752  
Epoch: [1][479/480] Data 12.011 (4.822) Elapsed 41m 7s (remain 0m 0s) Loss: 0.6915(0.6988) Grad: 2.3084  
EVAL: [0/60] Data 43.855 (43.855) Elapsed 0m 43s (remain 43m 14s) Loss: 0.7054(0.7054) 


Epoch 1 - avg_train_loss: 0.6988  avg_val_loss: 0.6928  time: 3158s
Epoch 1 - Score: 0.5274
Epoch 1 - Save Best Score: 0.5274 Model


EVAL: [59/60] Data 0.000 (11.406) Elapsed 11m 30s (remain 0m 0s) Loss: 0.6464(0.6928) 


Epoch 1 - Save Best Loss: 0.6928 Model


...

Epoch 24 - avg_train_loss: 0.6194  avg_val_loss: 0.5493  time: 214s
Epoch 24 - Score: 0.8167


EVAL: [59/60] Data 0.000 (0.586) Elapsed 0m 41s (remain 0m 0s) Loss: 0.3365(0.5493) 
Epoch: [25][0/480] Data 1.207 (1.207) Elapsed 0m 1s (remain 14m 24s) Loss: 0.7755(0.7755) Grad: 0.8721  
Epoch: [25][100/480] Data 0.001 (0.012) Elapsed 0m 37s (remain 2m 20s) Loss: 0.5530(0.6173) Grad: 0.5875  
Epoch: [25][200/480] Data 0.000 (0.007) Elapsed 1m 13s (remain 1m 41s) Loss: 0.5948(0.6236) Grad: 0.7512  
Epoch: [25][300/480] Data 0.000 (0.005) Elapsed 1m 48s (remain 1m 4s) Loss: 0.5972(0.6243) Grad: 1.2978  
Epoch: [25][400/480] Data 0.002 (0.004) Elapsed 2m 24s (remain 0m 28s) Loss: 0.6179(0.6228) Grad: 0.7811  
Epoch: [25][479/480] Data 0.000 (0.003) Elapsed 2m 51s (remain 0m 0s) Loss: 0.5644(0.6228) Grad: 0.6011  
EVAL: [0/60] Data 2.031 (2.031) Elapsed 0m 2s (remain 2m 6s) Loss: 0.6675(0.6675) 


Epoch 25 - avg_train_loss: 0.6228  avg_val_loss: 0.5355  time: 201s
Epoch 25 - Score: 0.8152


EVAL: [59/60] Data 0.000 (0.370) Elapsed 0m 28s (remain 0m 0s) Loss: 0.3956(0.5355) 


========== fold: 0 result ==========
Score: 0.8178
========== CV ==========
Score: 0.8178


In [25]:
models = get_models(CFG, mode='loss')
models_ = get_models(CFG, mode='score')
test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size*2, shuffle=False, 
                         num_workers=CFG.num_workers, pin_memory=True)
fold_preds = flip_inference(models, test_loader, device)
fold_preds_ = flip_inference(models_, test_loader, device)
preds = np.mean(fold_preds, axis=0)
preds_ = np.mean(fold_preds_, axis=0)
preds = probability_calibration(preds, beta)
preds_ = probability_calibration(preds_, beta)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [26]:
test = test[['id', 'target']]
test[CFG.target_col] = (preds + preds_) / 2
test.to_csv(OUTPUT_DIR+'submission.csv', index=False)
display(test)

,id,target
0,000bf832cae9ff1,0.024246
1,000c74cc71a1140,0.083529
2,000f5f9851161d3,0.077155
3,000f7499e95aba6,0.127361
4,00133ce6ec257f9,0.042043
...,...,...
39990,fff701540fb8aad,0.070072
39991,fff9d92a46ea99a,0.090400
39992,fffa40260ffe43d,0.050091
39993,fffbc626c9d269f,0.034362
